In [14]:
'''
functions which will filter out invalid moves from logits 
'''

'\nfunctions which will filter out invalid moves from logits \n'

In [15]:
import tensorflow as tf

In [30]:
class ValidationFunctions:
    def getInvalidAttacks(moveDictList):
        index = 0
        invalidMoves = []
        for subDict in moveDictList:
            try:
                if subDict['disabled']:
                    invalidMoves.append(index)
            except KeyError:
                return [1,2,3,4,5,6,7,8,9]
            index +=1
        numMoves = len(moveDictList)
        for i in range(4 - numMoves):
            invalidMoves.append(3 - i)
        return invalidMoves
    def getFaintedPkmn(statusDict):
        names = []
        for i in statusDict.keys():
            if statusDict[i] == 'fnt':
                names.append(i)
        return names 
    def getInvalidSwitches(statusDict, activePkmn, canonicalNames):
        names = ValidationFunctions.getFaintedPkmn(statusDict)
        if activePkmn != '':
            names.append(activePkmn)
        invalidMoves = []
        for i in names:
            name = i
            invalidMoves.append(canonicalNames.index(name) + 4)
        return invalidMoves
    def getInvalidMoves(playerDict, canonicalNames):
        invalidAttacks = []
        if 'current' in playerDict:
            moveDictList = playerDict['current']
            invalidAttacks = ValidationFunctions.getInvalidAttacks(moveDictList)
        fntedMoves = ValidationFunctions.getInvalidSwitches(playerDict['status'], playerDict['active'], canonicalNames)
        if 'forceSwitch' in playerDict:
            for i in range(4):
                invalidAttacks.append(i)
        if 'maybeTrapped' in playerDict:
            for i in range(6):
                invalidAttacks.append(i + 4)
        return set(invalidAttacks + fntedMoves)
    def getNameDict(canonicalNames, currentNames):
        '''
        map 1,2,3,4,5,6
        to  2,4,6,3,5,1
        1 would be 6
        2 would be 6
        3 would be 4
        4 would 2
        5 would be 5
        6 would be 3
        '''
        nameDict = {}
        for i in range(len(canonicalNames)):
            j = i + 1
            nameDict[j] = currentNames.index(canonicalNames[i]) + 1
        return nameDict
    def testMove(playerDict, canonicalNames):
        invalidMoves = ValidationFunctions.getInvalidMoves(playerDict, canonicalNames)
        print("###################### Invalid Moves ####################")
        print(invalidMoves)
        while True:
            i = random.randint(0, 9) 
            if i not in invalidMoves:
                return i
    def filterLogits(action_probs, playerDict, canonicalNames):
        invalidIndices = ValidationFunctions.getInvalidMoves(playerDict, canonicalNames)
        if len(invalidIndices) == 0:
            return action_probs
        initialshape = tf.shape(action_probs)
        indices = []
        values = []
        shape = [1, 10]
        shape = tf.cast(shape, tf.int64)
        action_probs = tf.reshape(action_probs, shape)
        invalidIndices = sorted(invalidIndices)
        for action in invalidIndices:
            indices.append([0, action])
            values.append(-float(action_probs[0, action]))
        indices = np.array(indices, dtype = np.int64)
        values = np.array(values, dtype = float)
        delta = tf.SparseTensor(indices, values, shape)
        action_probs = tf.cast(action_probs, dtype = tf.float32)
        delta = tf.cast(delta, dtype = tf.float32)
        action_probs = action_probs + tf.sparse.to_dense(delta)
        action_probs = tf.reshape(action_probs, initialshape)
        return action_probs
    def normalizeLogits(action_probs):
        action_probs_reshaped = tf.reshape(action_probs, [10])
        action_probs_numpy = action_probs_reshaped.numpy()
        values = []
        indices = []
        total = 0
        for num in action_probs_numpy:
            total += float(num)
        counter = 0
        for num in action_probs_numpy:
            action_probs_numpy[counter] = num/total
            counter += 1
        counter = 0
        for num in action_probs_numpy:
            values.append(float(action_probs_numpy[counter]) - float(action_probs[0, counter]))
            indices.append([0, counter])
            counter += 1
        delta = tf.SparseTensor(indices, values, [1, 10])
        action_probs = action_probs + tf.sparse.to_dense(delta)
        return action_probs
    def processActionProbs(action_probs, playerDict, canonicalNames):
        action_probs_t = ValidationFunctions.filterLogits(action_probs, playerDict, canonicalNames)
        action_logits_t_used = ValidationFunctions.normalizeLogits(action_probs_t)
        return action_logits_t_used
    def getMaxDamageMove(playerDict):
        maxMove = 99
        prevPower = 0
        try:
            moves = playerDict['moves'][playerDict['active']]
            for i in range(len(moves)):
                moveList = Databases.getMoveList(moves[i])
                if moveList[0] > prevPower:
                    prevPower = moveList[0]
                    maxMove = i
            if maxMove != 99:
                return maxMove
            else:
                return None
        except KeyError:
            return None     